In [6]:
# torch packets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# torch vision 
from torchvision import datasets, transforms

## prepare  MNIST dataset 

In [7]:
mnist_path = 'DATA'
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])


train_set = datasets.MNIST(root=mnist_path, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=mnist_path, train=False, transform=trans)
train_set_flat = train_set.data.view(60000, 28 * 28 )

## 定义网络

In [25]:
class mnist_net(nn.Module):
    def __init__(self):
        super(mnist_net, self).__init__() # 为啥？
        self.fc1 = nn.Linear(28* 28 ,  500)
        self.fc2 = nn.Linear(500, 32)
        self.fc3 = nn.Linear(32, 10)
        pass
    
    
    def forward(self, x):
        x = F.relu( self.fc1(x)  )
        x = F.relu( self.fc2(x)  )
        out = self.fc3(x)
        return out 
        pass 
    pass


model = mnist_net()

## 定义 优化器

In [31]:
opt = optim.Adam(model.parameters() ) # 其他可以使用默认参数

ceriation = nn.CrossEntropyLoss()

batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)


## 训练

In [ ]:
for epoch in xrange(10):
    ave_loss = 0 # 干什么的？ 
    for batch_index, (x, target) in enumerate(train_loader):
        output = model.forward(x) 
        loss = ceriation(output , target)
        pass
    
    pass